1. 불러오기
- 2_token.ipynb에서 저장한 파일을 불러온다.

In [1]:
#라이브러리 import
import pickle

In [5]:
with open('token.pickle', 'rb') as f:
    token_list = pickle.load(f)
len(token_list)

132

In [6]:
token_list[0]

['어리',
 'ㄴ',
 '햇살',
 '아래',
 '서',
 '뛰어놀',
 '곤',
 '하',
 '었',
 '었',
 '더',
 'ㄴ',
 '가쁘',
 'ㄴ',
 '숨결',
 '굽이지',
 'ㄴ',
 '골목',
 '지나',
 '기',
 '를',
 '을',
 '따르',
 '아',
 '가보',
 '면',
 '같',
 '은',
 '기억',
 '어른',
 '이',
 '란',
 '시간',
 '은',
 '아직',
 '어색',
 '하',
 '게',
 '도',
 '나',
 '를',
 '채우',
 '어',
 '많',
 '은',
 '것',
 '이',
 '변하',
 '었',
 '다',
 '하',
 '어',
 '여기',
 'ㄴ',
 '그대로',
 '인걸',
 'you',
 '’',
 'll',
 'feel',
 'the',
 'same',
 '땀',
 '에',
 '젖',
 '어',
 '놀',
 '았',
 '던',
 '우리',
 '는',
 '너와',
 '낳',
 'ㄴ',
 '이젠',
 '돌아가',
 'ㄹ',
 '순',
 '없',
 '지만',
 '낡',
 '아',
 '진',
 '서랍',
 '속',
 '에서',
 '작',
 '았',
 '더',
 'ㄴ',
 '서',
 '롤',
 '기억',
 '하',
 '어',
 'when',
 'i',
 '’',
 'm',
 'far',
 'from',
 'home',
 'always',
 '떠오',
 'ㄹ라',
 '나',
 '는',
 '아직',
 '도',
 '반짝이',
 '더',
 'ㄴ',
 '네',
 '두',
 '눈',
 '마주',
 '보',
 '더',
 'ㄴ',
 '그림자',
 '마주',
 '보',
 '더',
 'ㄴ',
 '우리',
 '는',
 '여기',
 'still',
 'same',
 '시간',
 '은',
 '언제나',
 '날',
 '울리',
 '는',
 '존재',
 '지만',
 '놓',
 '지',
 '를',
 '못하',
 '어',
 '많',
 '은',
 '것',
 '이',
 '더',
 '지나',
 'ㄹ',
 '때',
 '에는',
 '여기',


2. 임베딩 - fasttext
- 토큰화한 가사를 임베딩한다. 간단히 fastText를 사용해보려 한다.

In [7]:
#라이브러리 import
from gensim.models import FastText

In [8]:
#fasttest에 넣어보기
ft_model = FastText(token_list, window=5, workers=4, sg=1)

In [9]:
#임베딩되어 비슷한 단어들 확인하기, 너무 단어 수가 적어서 그런지 높은 수치는 나오지 않았다.
print(ft_model.predict_output_word('오늘'))
print(ft_model.predict_output_word('이별'))
print(ft_model.predict_output_word('눈'))

[('오', 0.0019067173), ('아도', 0.001772606), ('말하', 0.0017076706), ('밝', 0.001703193), ('좋', 0.0016874506), ('여기', 0.0016272629), ('너무', 0.001619117), ('서', 0.0016109733), ('ㄹ까', 0.001606439), ('대', 0.0015963953)]
[('bye', 0.0018432686), ('서', 0.0018009212), ('오늘', 0.0017380161), ('여기', 0.0017361784), ('눈', 0.0017331064), ('지나가', 0.00168582), ('전', 0.0016804836), ('모든', 0.0016725119), ('지금', 0.0016245942), ('요', 0.001614327)]
[('눈', 0.0026489075), ('앞', 0.0021715013), ('두', 0.0019035005), ('전', 0.0018851173), ('서', 0.001855663), ('다', 0.0017894548), ('집', 0.0017831034), ('ㄹ까', 0.0017797357), ('지나가', 0.0017787481), ('우우', 0.001773215)]


3. 간단한 모델링
- 이전에 진행한 fasttest의 결과를 어떻게 모델에 집어넣어야 할지 몰라서, 다른 방식을 찾아 진행해보았다. lecture note의 코드를 참조하였다.

In [10]:
#라이브러리 import
import tensorflow as tf
from tensorflow.keras import preprocessing
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import io
import sys
import random

In [11]:
#파일을 열고 말뭉치 길이 출력)
with io.open('token.txt', encoding='utf-8') as f:
    text = f.read()
print('corpus length:', len(text))

#불러온 파일 처리
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

corpus length: 88022
total chars: 840


In [12]:
#문자열 크기 정렬해서 벡터화(임베딩)
maxlen = 20
step = 7

sentences = []
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

nb sequences: 12572


C:\Users\zekca\AppData\Local\Temp/ipykernel_13440/665535834.py:13: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
C:\Users\zekca\AppData\Local\Temp/ipykernel_13440/665535834.py:14: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = np.zeros((len(sentences), len(chars)), dtype=np.bool)


In [14]:
#LSTM 모델 빌드
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))
optimizer = RMSprop(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics='accuracy')

In [15]:
#샘플 문장 생성
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

#Epoch가 끝날 때마다 sample 문장을 생성
def on_epoch_end(epoch, _):
    print()
    print('----- 예시 문장 생성: %d' % epoch)
    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(300):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [17]:
model.fit(x, y,
          batch_size=128,
          epochs=100,
          callbacks=[print_callback])

Epoch 1/100
98/99 [============================>.] - ETA: 0s - loss: 0.7554 - accuracy: 0.8382
----- 예시 문장 생성: 0
----- diversity: 0.2
----- Generating with seed: "ear don ' t you come"
ear don ' t you come stou withith anki gar thtelt to me withtat to me withtant tour talkte tou to stou to stou withith 이렇 게 하 고 가 가 아 우우 우 우 우 우 우 우 우 지 않 게 으로 그데 나 에 겐 바보 ㄹ 것 이 안 나 가 아 또 말 야 이 안 아 주 어요 안 아 아 주 어요 안 아 아 주 어요 안 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아
----- diversity: 0.5
----- Generating with seed: "ear don ' t you come"
ear don ' t you come 없 고 stou 이렇 게 ) 운 이 되 지 마 어 너 의 사람 하 어 보 아 우우 우 우 우 우 우 우 지 네 게 우우 ㄴ 그 로 오오 마 우리 우 지 어 보 에 게 이 어 가 터 조은 하 어 주 어요 버리 어 가 아 주 어요 안 아 아 주 어요 안 아 아 주 어요 안 아 아 주 어요 안 아 아 주 어요 안 아 아 주 어요 안 아 아 주 어요 안 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 
----- diversity: 1.0
----- Generating with seed: "ear don ' t you come"
ear don ' t you come ston i t

C:\Users\zekca\AppData\Local\Temp/ipykernel_13440/2550270886.py:4: RuntimeWarning: divide by zero encountered in log
  preds = np.log(preds) / temperature


 만 보이 어느 믿 김o은 지 연를 만 문어 만 보 어 면 게 날 아 내 게 하 서 는 네 생늘 의 지 않 는 데 ( ㄴ타 고 스 어 혼자무 ㄹ 나 는 지금 은 길 이 줄 두 비 르 ㄹ 마억 다른 어 ㄹ 보 ㄹ 때 마 도 나 의 마르 ( 나 는 너 에 서이하 어 있 지 도 네 때 주 다시 느 메챙 나 의 사젠 롭 진i 어도 채 을리 에 u그가 차 은 버해 이 가 고 ㄹ 말 도 없아는 바늘 날 
99/99 [==============================] - 44s 444ms/step - loss: 0.1487 - accuracy: 0.9715
Epoch 19/100
98/99 [============================>.] - ETA: 0s - loss: 0.1374 - accuracy: 0.9723
----- 예시 문장 생성: 18
----- diversity: 0.2
----- Generating with seed: "도 문득 사람 들 과 인사 를 하 고"
도 문득 사람 들 과 인사 를 하 고 짓 은 날 은 날 그 ㄴ 우 우 우 우 우 지 않 다 세요 고 에 두 세요 그대 니 수다가 없겁 어도 나 는 아 또 하 어 그대 와 ㄴ 내 게 내 눈치 ㄴ 면금 나 는 바i람 날 그대 와 같이 은 이 세요 아 주 그 오늘 에 그대 가 그만 와어 하 어 이것 었 지 않 다 뜨t he 하 만 너 도 수l하 있 었 는 다가 도 새아 니 니 가 아 내 게 , al 그대 는 것 은 두 ㄴ 노래 이 ㄹ지 어 주 어 내 게 n i 처럼 의 마음 만 너 의 비리 를 기 고 있 은 니 가 만 ( 그러 가 내 고 싶 은 니 가 만나 ㄹ지 그 그대 
----- diversity: 0.5
----- Generating with seed: "도 문득 사람 들 과 인사 를 하 고"
도 문득 사람 들 과 인사 를 하 고 짓 은 날 은 날 그대 는 맘 은 말 고 못 우리 는 진미 이 얼 너 의 너 는 없 지만 나 는 손 은 것 을 혼자리 는 내 이 쏟 라 는 우리 는 것 이 

In [18]:
#평가
model.evaluate(x, y)

393/393 [==============================] - 4s 8ms/step - loss: 0.0133 - accuracy: 0.9962


[0.013250628486275673, 0.9961819648742676]

4. 피드백
- 형태소 기반 토큰화때문인지 혹은 모델링 때문인지 혹은 기타 다른 것 때문인지 만족스럽지 않다. 정확도는 높고 loss는 낮아 '잘 구현된 모델'이 맞지만, 사용가능한 수준인가? 물었을 때 불가능한 편이다. 따라서 다른 방식으로 다시 진행해보려 한다.